<a href="https://colab.research.google.com/github/hazmicokro/Classification-Cardiovascular-using-eXtreme-Gradient-Boosting-XGB/blob/main/final_project_ClassificationOfCardiovascularDiseaseUsingXGBoost_M_Hazmi_Cokro_Mandiri.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%html
<marquee style='width: 100%; color: #ff7b54;'>
  <b>Final Project - DLB 2020 - Classification of Cardiovascular Disease Using eXtreme Gradient Boosting (XGBoost) - Hazmi Cokro</b>
</marquee>

**FINAL PROJECT | DLB 2020** \
**Tema** : Classification using XGBoost \
**Author** : M Hazmi Cokro M \
**Judul** : **Classification of Cardiovascular Disease using eXtreme Gradient Boosting (XGBoost)** 


---


**Dataset** : [Cardiovascular Disease dataset](https://www.kaggle.com/sulianova/cardiovascular-disease-dataset) \
**Format Data** : CSV \
**Total Data** : 70.000  \
**Features** : 11 Feature + 1 Target (2 Class = Cardio (1) , Non-Cardio (0))


#**[0] Library**

In [1]:
# u/ mengabaikan warnings / peringatan
import warnings
warnings.filterwarnings("ignore")

# u/ menghubungkan ke gdrive
from google.colab import drive
from google.colab import files

# u/ mengolah data csv
import os
import numpy as np
import pandas as pd

# u/ plot visualisasi data
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots 

# u/ preprocessing data
from sklearn.model_selection import train_test_split
from sklearn.preprocessing   import RobustScaler,MinMaxScaler

# u/ training dan evaluasi model
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix,precision_score,recall_score,f1_score

# u/ saving model
import joblib

Mount / Menghubungkan Gdrive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


Mengatur lokasi kerja (workspace)

In [ ]:
%cd "/content/drive/MyDrive/final_project_ClassificationOfCardiovascularDiseaseUsingXGBoost_M Hazmi Cokro Mandiri"

/content/drive/MyDrive/final_project_ClassificationOfCardiovascularDiseaseUsingXGBoost_M Hazmi Cokro Mandiri


Mengatur letak konfigurasi kaggle

In [ ]:
os.environ['KAGGLE_CONFIG_DIR'] = "/content/drive/MyDrive/final_project_ClassificationOfCardiovascularDiseaseUsingXGBoost_M Hazmi Cokro Mandiri"

Upload File api dari kaggle berformat .json

In [ ]:
files.upload()
print("Sukses Upload kaggle.json")

Saving kaggle.json to kaggle.json
Sukses Upload kaggle.json


Mengatur permission dari kaggle json u/ dapat mendownload dataset

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle

!chmod 600 /root/.kaggle/kaggle.json

kaggle.json


#**[1] Prepare Dataset**

###**[1.1] Download Dataset**
Download dan Ekstrak Data

Download dataset bernama Cardiovascular disease dataset dari kaggle

In [ ]:
!kaggle datasets download -d sulianova/cardiovascular-disease-dataset

  0% 0.00/742k [00:00<?, ?B/s]
100% 742k/742k [00:00<00:00, 49.6MB/s]


Unzip / Ekstrak dataset

In [ ]:
!unzip cardiovascular-disease-dataset.zip

Archive:  cardiovascular-disease-dataset.zip
  inflating: cardio_train.csv        


###**[1.2] Load Dataset**
Load Dataset berformat CSV

In [ ]:
# Load data dengan pandas, separator titik koma ; , dan index kolom menggunakan kolom id
df_cardio = pd.read_csv('cardio_train.csv', sep=';', index_col="id")
df_cardio.head()

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
id,,,,,,,,,,,,
0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,18857,1,165,64.0,130,70,3,1,0,0,0,1
3,17623,2,169,82.0,150,100,1,1,0,0,1,1
4,17474,1,156,56.0,100,60,1,1,0,0,0,0


###**[1.3] Shape Dataset**
Jumlah Baris dan Kolom

In [ ]:
print("Jumlah Baris : {}".format(df_cardio.shape[0]))
print("Jumlah Kolom : {}".format(df_cardio.shape[1]))

Jumlah Baris : 70000
Jumlah Kolom : 12


###**[1.4] Info Dataset**
Nama Kolom, jumlah dan tipe data tiap kolom

| Features  | Tipe Features | Description |
| ------------- | ------------- | ------------- |
| age | Objective feature | in days |
| gender | Objective feature | 1- women , 2-men |
|height | Objective feature | in cm |
|weight | Objective feature|in Kg|
|ap_hi / Systolic blood pressure | Examination Feature | int |
|ap_lo / Diastolic blood pressure | Examination Feature | int |
|cholesterol | Examination Feature | 1: normal, 2: above normal, 3: well above normal |
|gluc / Glucose| Examination Feature |1: normal, 2: above normal, 3: well above normal |
|smoke / Smoking|Subjective Feature |binary |
|alco / Alcohol intake | Subjective Feature | binary |
|active / Physical activity | Subjective Feature | binary |
|cardio | Presence or absence of cardiovascular disease | Target Variable |binary |

In [ ]:
df_cardio.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 70000 entries, 0 to 99999
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   age          70000 non-null  int64  
 1   gender       70000 non-null  int64  
 2   height       70000 non-null  int64  
 3   weight       70000 non-null  float64
 4   ap_hi        70000 non-null  int64  
 5   ap_lo        70000 non-null  int64  
 6   cholesterol  70000 non-null  int64  
 7   gluc         70000 non-null  int64  
 8   smoke        70000 non-null  int64  
 9   alco         70000 non-null  int64  
 10  active       70000 non-null  int64  
 11  cardio       70000 non-null  int64  
dtypes: float64(1), int64(11)
memory usage: 6.9 MB


###**[1.5] Check Null / NaN Value**
Mengecek nilai null / kosong pada data

In [ ]:
# Pada dataset terlihat tidak terdapat nilai null pada data ini
df_cardio.isna().sum()

age            0
gender         0
height         0
weight         0
ap_hi          0
ap_lo          0
cholesterol    0
gluc           0
smoke          0
alco           0
active         0
cardio         0
dtype: int64

###**[1.6] Rename Kolom Dataset**
Merubah nama kolom dataset

In [ ]:
# Memudahkan dalam hal pembacaan kolom dan kejelasan kolom
df_cardio.rename(columns={'ap_hi': 'systolic', 'ap_lo': 'diastolic',
                                              'gluc': 'glucose', 'alco': 'alcohol',
                                              'cardio': 'cardiovascular'}, inplace=True)

###**[1.7] Convert Format Kolom Age**
Mengkonversikan kolom age menjadi format tahun

In [ ]:
# Mengkonversikan kolom age dari format hari menjadi format tahun (365)
df_cardio.age = np.round(df_cardio.age/365,decimals=0)
df_cardio.head()

,age,gender,height,weight,systolic,diastolic,cholesterol,glucose,smoke,alcohol,active,cardiovascular
id,,,,,,,,,,,,
0,50.0,2,168,62.0,110,80,1,1,0,0,1,0
1,55.0,1,156,85.0,140,90,3,1,0,0,1,1
2,52.0,1,165,64.0,130,70,3,1,0,0,0,1
3,48.0,2,169,82.0,150,100,1,1,0,0,1,1
4,48.0,1,156,56.0,100,60,1,1,0,0,0,0


###**[1.8] Deskripsi Dataset**
Deskripsi dataset dan mengeksplore dataset

In [ ]:
df_cardio.describe().round()

,age,gender,height,weight,systolic,diastolic,cholesterol,glucose,smoke,alcohol,active,cardiovascular
count,70000.0,70000.0,70000.0,70000.0,70000.0,70000.0,70000.0,70000.0,70000.0,70000.0,70000.0,70000.0
mean,53.0,1.0,164.0,74.0,129.0,97.0,1.0,1.0,0.0,0.0,1.0,0.0
std,7.0,0.0,8.0,14.0,154.0,188.0,1.0,1.0,0.0,0.0,0.0,1.0
min,30.0,1.0,55.0,10.0,-150.0,-70.0,1.0,1.0,0.0,0.0,0.0,0.0
25%,48.0,1.0,159.0,65.0,120.0,80.0,1.0,1.0,0.0,0.0,1.0,0.0
50%,54.0,1.0,165.0,72.0,120.0,80.0,1.0,1.0,0.0,0.0,1.0,0.0
75%,58.0,2.0,170.0,82.0,140.0,90.0,2.0,1.0,0.0,0.0,1.0,1.0
max,65.0,2.0,250.0,200.0,16020.0,11000.0,3.0,3.0,1.0,1.0,1.0,1.0


Setelah saya mengeksplore dataset tersebut dan melalui penjabaran deskripsi [1.8] dataset diatas saya mendapat beberapa informasi antara lain :
1. Usia terendah penderita penyakit ini berumur 30th dan usia tertingginya 65th maka dapat disimpulkan bahwa koresponden penelitian ini tidak melibatkan anak-anak
2. Berdasarkan referensi [Heart.org](https://www.heart.org/en/health-topics/high-blood-pressure/understanding-blood-pressure-readings) dikatakan bahwa penderita dengan tekanan darah tinggi (systolic) lebih dari 180mmHg atau tekanan darah rendah (diastolic) lebih dari 120mmHg harus sesegera mungkin ditangani dokter (parah). 
Namun pada data ini nilai tertinggi berada pada 16020mmHg dan 110000mmHg dimana hal tersebut aneh serta pada nilai minimumnya yang berada pada nilai negatif sehingga dapat dikatakan sebagai data pencilan / outlier
3. Pada kolom height ketinggian Minimum dan Maksimum terlihat salah. Meskipun juga dapat merepresentasikan manusia terkecil dan tertinggi : 55 cm dan 250 cm, namun tetap tampak seperti kesalahan data (Error).

#**[2] Preprocessing**

###**[2.1] Remove Duplicate Data**
Menghapus data ganda / duplicate

In [ ]:
# Data duplicate berjumlah 3816 baris
print("Total Data Duplicate = ",df_cardio.duplicated().sum(), "\n\n")
df_cardio[df_cardio.duplicated()].sort_values(by=['gender', 'height', 'weight'], ascending= False)

Total Data Duplicate =  3816 




,age,gender,height,weight,systolic,diastolic,cholesterol,glucose,smoke,alcohol,active,cardiovascular
id,,,,,,,,,,,,
34489,46.0,2,183,80.0,120,80,1,1,0,0,0,0
87772,58.0,2,180,100.0,120,80,1,1,0,0,1,0
42239,56.0,2,180,80.0,150,90,1,1,0,0,1,1
76280,56.0,2,180,80.0,120,80,1,1,0,0,1,1
77664,40.0,2,180,80.0,120,80,1,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
82518,58.0,1,147,64.0,110,80,1,1,0,0,1,0
96091,58.0,1,146,52.0,120,80,1,1,0,0,1,0
90867,64.0,1,145,50.0,120,80,1,1,0,0,1,1


In [ ]:
# Menghapus baris yang ganda / duplicate
df_cardio.drop_duplicates(inplace=True)
df_cardio.head()

,age,gender,height,weight,systolic,diastolic,cholesterol,glucose,smoke,alcohol,active,cardiovascular
id,,,,,,,,,,,,
0,50.0,2,168,62.0,110,80,1,1,0,0,1,0
1,55.0,1,156,85.0,140,90,3,1,0,0,1,1
2,52.0,1,165,64.0,130,70,3,1,0,0,0,1
3,48.0,2,169,82.0,150,100,1,1,0,0,1,1
4,48.0,1,156,56.0,100,60,1,1,0,0,0,0


In [ ]:
print("Jumlah Baris : {}".format(df_cardio.shape[0]))

Jumlah Baris : 66184


###**[2.2] Systolic dan Diastolic**
BoxPlot dan Remove Outliers

![Blood Pressure](https://www.heart.org/-/media/health-topics-images/hbp/blood-pressure-readings-chart-english.jpg?la=en)

####**[2.2.1] BoxPlot Systolic & Diastolic**
Visualisasi Boxplot Systolic dan Diastolic

In [ ]:
plt.rcParams['figure.figsize'] = [20, 12]
plt.rcParams['font.size'] = 12

In [ ]:
fig2 = go.Figure()

fig2.add_trace(go.Box(x=df_cardio['systolic'], name = 'Systolic', boxpoints='outliers',))
fig2.add_trace(go.Box(x=df_cardio['diastolic'], name = 'Diastolic', boxpoints='outliers',))

fig2.update_layout(title_text="BoxPlot Kolom systolic dan diastolic dengan Outliers")
fig2.show()

####**[2.2.2] Remove Outlier S & D**
Remove Outlier dari Systolic dan Diastolic

Tidak semua semua outlier di luar kuartil ke-3 diremove / hapus karena pasien dengan tekanan darah >180 pada systolic maupun >120 pada diastolic dapat mengalami krisis hipertensi. Maka dari itu saya ambil patokan untuk pasien krisis hipertensi di 210 systolic dan 150 diastolic.

Sehingga didapatkan batas 90 - 210 systolic dan 65 - 150 diastolic

In [ ]:
df_cardio = df_cardio[~((df_cardio['systolic'] >= 210) | (df_cardio['systolic'] <= 90) 
                                   | (df_cardio['diastolic'] >= 150) | (df_cardio['diastolic'] <= 65))]

In [ ]:
fig3 = go.Figure()

fig3.add_trace(go.Box(x=df_cardio['systolic'], name = 'Systolic', boxpoints='outliers',))
fig3.add_trace(go.Box(x=df_cardio['diastolic'], name = 'Diastolic', boxpoints='outliers',))

fig3.update_layout(title_text="BoxPlot Kolom systolic dan diastolic setelah Remove Outliers")
fig3.write_image("boxplot2.svg")
fig3.show()

###**[2.3] Height dan Weight**
Boxplot dan Remove Outlier pada kolom Height dan Weight

####**[2.3.1] BoxPlot Height & Weight**
Visualisasi Boxplot Height dan Weight

In [ ]:
fig = go.Figure()

fig.add_trace(go.Box(x=df_cardio['height'], name = 'Height', boxpoints='outliers',))
fig.add_trace(go.Box(x=df_cardio['weight'], name = 'Weight', boxpoints='outliers',))

fig.update_layout(title_text="BoxPlot Kolom Weight dan Height dengan Outliers")
fig.show()

####**[2.3.2] Remove Outlier H & W**
Remove Outlier pada kolom Height dan Weight

Sama halnya dengan systolic dan diastolic tidak semua outlier dihapuskan 

In [ ]:
df_cardio = df_cardio[(df_cardio['height'] > 143) & (df_cardio['height'] < 210)]
df_cardio = df_cardio[~(df_cardio['weight'] < 40)]

In [ ]:
fig4 = go.Figure()

fig4.add_trace(go.Box(x=df_cardio['height'], name = 'Height', boxpoints='outliers',))
fig4.add_trace(go.Box(x=df_cardio['weight'], name = 'Weight', boxpoints='outliers',))

fig4.update_layout(title_text="BoxPlot Kolom Weight dan Height setelah Remove Outliers")
fig4.show()

###**[2.4] Split Dataset**
Pembagian Dataset menjadi 80% Train dan 20% Test \
Pembagian tsb didasari dari [Pareto Principle](https://en.wikipedia.org/wiki/Pareto_principle) 

In [ ]:
# X feature dari data dan Y sebagai target label
X = df_cardio.drop(['cardiovascular'], axis = 1).copy()
y = df_cardio['cardiovascular']

# Split Data berdasarkan Parrot Principle (80:20)
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size = 0.20, random_state = 88)

###**[2.5] Rescaling Dataset**
Rescale dataset menggunakan Robust Scaler dan MinMax Scaler



---

Sebelum itu proses scaling ini terlebih dahulu dilakukan splitting data karena Jika scaling digunakan sebelum splitting data (train_test_split) maka akan terjadi kebocoran data. Untuk menanggulangi itu maka dilakukanlah scaling setelah splitting data (train_test_split)

Penjelasan memakai 2 Scaler yaitu Robust Scaler dan MinMax Scaler :

- MinMax Scaler : \
Proses scaling yang akan mengubah setiap nilai di kolom secara proporsional dalam rentang [0,1] .

- Robust Scaler : \
Sebagai alternatif proses scaling ketika terdapat data pencilan / outlier.

In [ ]:
#RobustScaler
rs = RobustScaler()

#Diastolic - RobustScaler
X_train['diastolic'] = rs.fit_transform(X_train[['diastolic']].values)
X_test['diastolic'] = rs.fit_transform(X_test[['diastolic']].values)

#Systolic - RobustScaler
X_train['systolic'] = rs.fit_transform(X_train[['systolic']].values)
X_test['systolic'] = rs.fit_transform(X_test[['systolic']].values)

#Weight - RobustScaler
X_train['weight'] = rs.fit_transform(X_train[['weight']].values)
X_test['weight'] = rs.fit_transform(X_test[['weight']].values)

#MinMaxScaler
mms = MinMaxScaler()

#Age - MinMaxScaler
X_train['age'] = mms.fit_transform(X_train[['age']].values)
X_test['age'] = mms.fit_transform(X_test[['age']].values)

#Height - MinMaxScaler 
X_train['height'] = mms.fit_transform(X_train[['height']].values)
X_test['height'] = mms.fit_transform(X_test[['height']].values)

#**[3] Modelling XGBoost**
Model eXtreme Gradient Boosting Classifier

###**[3.1] Model XGBoost Sebelum Tuning**
Training Data dengan Model XGBoost sebelum dilakukan Hyperparameter Tuning

In [ ]:
# Deklarasi model XGBoost yaitu eXtreme Gradient Boosting Classifier
xgb_model = XGBClassifier()
xgb_model.fit(X_train,y_train)

ypred_xgb = xgb_model.predict(X_test)

#Classification Report
print("XGBoost Model - Classification Report \n")
print(classification_report(y_test, ypred_xgb))

#Confusion Matrix
print("XGBoost Model - Confusion Matrix Report \n")
print(confusion_matrix(y_test, ypred_xgb))

XGBoost Model - Classification Report 

              precision    recall  f1-score   support

           0       0.68      0.79      0.73      5822
           1       0.78      0.67      0.72      6464

    accuracy                           0.73     12286
   macro avg       0.73      0.73      0.73     12286
weighted avg       0.73      0.73      0.73     12286

XGBoost Model - Classification Report 

[[4608 1214]
 [2151 4313]]


In [ ]:
cm = confusion_matrix(y_test, ypred_xgb)
true_positive = cm[0][0]
false_positive = cm[0][1]
false_negative = cm[1][0]
true_negative = cm[1][1]

Precision = true_positive/(true_positive+false_positive)
print("Nilai Precision Model : ",Precision)

Recall= true_positive/(true_positive+false_negative)
print("Nilai Recall Model  : ",Recall)

F1_Score = 2*(Recall * Precision) / (Recall + Precision)
print("Nilai F1-Score Model  : ",F1_Score)

Nilai Precision Model :  0.7914805908622466
Nilai Recall Model  :  0.681757656458056
Nilai F1-Score Model  :  0.7325331849614498


#**[4] Hyperparameter Tuning XGBoost**
Hyperparameter Tuning Model eXtreme Gradient Boosting dengan GridSearchCV

###**[4.1] Tuning pada XGBClassifier**
Hyperparameter Tuning dengan GridSearchCV pada XGBClassifier

In [ ]:
#Deklarasi Model
xgb_model = XGBClassifier()

# Parameter untuk diHparams Tuning
param_grid = {
    'learning_rate': [0.1, 0.2],
    'max_depth': [6],
    'min_child_weight': [2],
    'subsample': [0.7],
    'n_estimators': [100, 300, 600]
}

# Proses Tuning Menggunakan GridSearchCV
xgb_tuning = GridSearchCV(xgb_model, param_grid, scoring='accuracy', cv=None, n_jobs=1)
xgb_tuning.fit(X_train, y_train)

# Hasil Parameter Terbaik
print("Best Parameter dari XGBoost : {}".format(xgb_tuning.best_params_))

Best Parameter dari XGBoost : {'learning_rate': 0.1, 'max_depth': 6, 'min_child_weight': 2, 'n_estimators': 100, 'subsample': 0.7}


###**[4.2] Model XGBoost Setelah Tuning**
Model XGBoost dengan Best Parameter dari proses Hyperparameter Tuning

In [ ]:
# Testing Model
ypred_xgb_tuning = xgb_tuning.predict(X_test)

# Classification Report
print("XGBoost Model Setelah HParams Tuning - Classification Report \n")
print(classification_report(y_test, ypred_xgb_tuning))

# Confusion Matrix
print("XGBoost Model Setelah HParams Tuning - Confusion Matrix \n")
print(confusion_matrix(y_test, ypred_xgb_tuning))

XGBoost Model - Classification Report 

              precision    recall  f1-score   support

           0       0.70      0.77      0.73      5822
           1       0.77      0.71      0.74      6464

    accuracy                           0.74     12286
   macro avg       0.74      0.74      0.74     12286
weighted avg       0.74      0.74      0.74     12286

XGBoost Model - Confusion Matrix 

[[4456 1366]
 [1884 4580]]


In [ ]:
# Nilai Indikator Confusion Matrix
cm = confusion_matrix(y_test, ypred_xgb)
true_positive = cm[0][0]
false_positive = cm[0][1]
false_negative = cm[1][0]
true_negative = cm[1][1]

# Nilai Precision
Precision = true_positive/(true_positive+false_positive)
print("Nilai Precision Model : ",Precision)

# Nilai Recall
Recall= true_positive/(true_positive+false_negative)
print("Nilai Recall Model  : ",Recall)

# Nilai F1-Score
F1_Score = 2*(Recall * Precision) / (Recall + Precision)
print("Nilai F1-Score Model  : ",F1_Score)

Nilai Precision Model :  0.7667468223978015
Nilai Recall Model  :  0.7022180273714016
Nilai F1-Score Model  :  0.7330651120781673


#**[5] Evaluasi Model**
Melakukan Evaluasi Model

In [ ]:
print("Parameter Terbaik : ", xgb_tuning.best_params_)

Parameter Terbaik :  {'learning_rate': 0.1, 'max_depth': 6, 'min_child_weight': 2, 'n_estimators': 100, 'subsample': 0.7}


In [9]:
acc_score1 = accuracy_score(y_test,ypred_xgb)*100
prec_score1 = precision_score(y_test,ypred_xgb)*100
rec_score1 = recall_score(y_test,ypred_xgb)*100
f_score1 = f1_score(y_test,ypred_xgb)*100

acc_score = accuracy_score(y_test,ypred_xgb_tuning)*100
prec_score = precision_score(y_test,ypred_xgb_tuning)*100
rec_score = recall_score(y_test,ypred_xgb_tuning)*100
f_score = f1_score(y_test,ypred_xgb_tuning)*100

data = [("XGBoost",
             '{:.2}%'.format(acc_score1),
             '{:.2}%'.format(prec_score1),
             '{:.2}%'.format(rec_score1), 
             '{:.2}%'.format(f1_score1)),
            ("XGBoost Hyperparameter Tuning	",
             '{:.2}%'.format(acc_score),
             '{:.2}%'.format(prec_score),
             '{:.2}%'.format(rec_score), 
             '{:.2}%'.format(f1_score))]

XGBoost_Model_Final = pd.DataFrame.from_records(data, columns = ["Model",'Accuracy',"Precision_score","Recall","F1-Score"]) 
XGBoost_Model_Final

,Model,Accuracy,Precision_score,Recall,F1-Score
0,XGBoost,73.0%,79.1%,68.1%,73.2%
1,XGBoost Hyperparameter Tuning,74.0%,76.6%,70.2%,73.3%


Saving Model U/ Deployment berformat Joblib

In [ ]:
filename = 'cardio_XGBoost.joblib'
joblib.dump(xgb_tuning, filename)
print("Sukses Saving Model")

Sukses Saving Model
